In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch as th
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F
from tqdm import tqdm
import os

from utils import data, metrics
import Frequentist_main as FCNN
import Bayesian_main as BCNN
from Bayesian.BayesianCNN import BBBAlexNet
from Frequentist.FrequentistCNN import AlexNet

In [2]:
# Set the device
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
# Set the parameters
priors={
    'prior_mu': 0,
    'prior_sigma': 0.1,
    'posterior_mu_initial': (0, 0.1),  # (mean, std) normal_
    'posterior_rho_initial': (-5, 0.1),  # (mean, std) normal_
}

n_epochs = 100
lr_start = 0.001
num_workers = 4
valid_size = 0.2
batch_size = 256
beta_type = "Blundell" 

In [4]:
# Dataset and Dataloader
c10_trainset, c10_testset, c10_inputs, c10_outputs = data.getDataset('CIFAR10')
c10_train_loader, c10_valid_loader, c10_test_loader = data.getDataloader(
    c10_trainset, c10_testset, valid_size, batch_size, num_workers)

c100_trainset, c100_testset, c100_inputs, c100_outputs = data.getDataset('CIFAR100')
c100_train_loader, c100_valid_loader, c100_test_loader = data.getDataloader(
    c100_trainset, c100_testset, valid_size, batch_size, num_workers)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# BayesianCNN with softplus on CIFAR10
bc10_sp_net = BBBAlexNet(c10_outputs, c10_inputs, priors, activation_type='softplus').to(device)
bc10_sp_criterion = metrics.ELBO(len(c10_trainset)).to(device)
bc10_sp_optimizer = Adam(bc10_sp_net.parameters(), lr=lr_start)
bc10_sp_lr_sched = lr_scheduler.ReduceLROnPlateau(bc10_sp_optimizer, patience=6, verbose=True)
bc10_sp_valid_loss_max = np.Inf

ckpt_name = 'Bayesian/Models/bc10_sp.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    bc10_sp_net.load_state_dict(checkpoint['model_state_dict'])
    bc10_sp_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    bc10_sp_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    bc10_sp_valid_loss_max = checkpoint['valid_loss_max']
    start_epoch = checkpoint['epoch'] + 1  
    print('Model loaded from {}'.format(ckpt_name))
else:
    start_epoch = 0 

for epoch in tqdm(range(start_epoch, n_epochs)):  # loop over the dataset multiple times

    bc10_sp_train_loss, bc10_sp_train_acc, bc10_sp_train_kl = BCNN.train_model(bc10_sp_net, bc10_sp_optimizer, bc10_sp_criterion, c10_train_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_sp_valid_loss, bc10_sp_valid_acc = BCNN.validate_model(bc10_sp_net, bc10_sp_criterion, c10_valid_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_sp_lr_sched.step(bc10_sp_valid_loss)

    # save model if validation accuracy has increased
    if bc10_sp_valid_loss <= bc10_sp_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc10_sp_valid_loss_max, bc10_sp_valid_loss))
        th.save({
            'model_state_dict': bc10_sp_net.state_dict(),
            'optimizer_state_dict': bc10_sp_optimizer.state_dict(),
            'scheduler_state_dict': bc10_sp_lr_sched.state_dict(),
            'valid_loss_max': bc10_sp_valid_loss,
            'epoch': epoch
        }, ckpt_name)
        bc10_sp_valid_loss_max = bc10_sp_valid_loss

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc10_sp_train_loss, bc10_sp_train_acc, bc10_sp_valid_loss, bc10_sp_valid_acc, bc10_sp_train_kl))

# After all epochs are complete, evaluate the model on the test set
bc10_sp_test_loss, bc10_sp_test_acc = BCNN.validate_model(bc10_sp_net, bc10_sp_criterion, c10_test_loader, num_ens=1, beta_type=beta_type)
print('Test Loss: {:.4f} \tTest Accuracy: {:.4f}'.format(bc10_sp_test_loss, bc10_sp_test_acc))

  1%|          | 1/100 [03:03<5:03:32, 183.97s/it]

Validation loss decreased (inf --> 23235517.065625).  Saving model ...
Epoch: 0 	Training Loss: 3641796.2032 	Training Accuracy: 0.1523 	Validation Loss: 23235517.0656 	Validation Accuracy: 0.2140 	train_kl_div: 463689881.6815


  2%|▏         | 2/100 [06:07<4:59:40, 183.47s/it]

Validation loss decreased (23235517.065625 --> 21709061.231250).  Saving model ...
Epoch: 1 	Training Loss: 3038038.8322 	Training Accuracy: 0.2473 	Validation Loss: 21709061.2312 	Validation Accuracy: 0.2790 	train_kl_div: 434250893.8599


  3%|▎         | 3/100 [09:10<4:56:36, 183.47s/it]

...

Validation loss decreased (1994466.506641 --> 1932394.887402).  Saving model ...
Epoch: 59 	Training Loss: 295192.1342 	Training Accuracy: 0.6504 	Validation Loss: 1932394.8874 	Validation Accuracy: 0.5904 	train_kl_div: 37503652.6624


 61%|██████    | 61/100 [3:06:45<1:58:31, 182.36s/it]

Validation loss decreased (1932394.887402 --> 1869437.514453).  Saving model ...
Epoch: 60 	Training Loss: 286613.8096 	Training Accuracy: 0.6536 	Validation Loss: 1869437.5145 	Validation Accuracy: 0.5974 	train_kl_div: 36255027.7452


 62%|██████▏   | 62/100 [3:09:43<1:54:45, 181.19s/it]

Validation loss decreased (1869437.514453 --> 1807599.759473).  Saving model ...
Epoch: 61 	Training Loss: 278239.6242 	Training Accuracy: 0.6568 	Validation Loss: 1807599.7595 	Validation Accuracy: 0.6089 	train_kl_div: 35051239.4140


In [ ]:
#BayesianCNN with softplus on CIFAR100
bc100_sp_net = BBBAlexNet(c100_outputs, c100_inputs, priors, activation_type='softplus').to(device)
bc100_sp_criterion = metrics.ELBO(len(c100_trainset)).to(device)
bc100_sp_optimizer = Adam(bc100_sp_net.parameters(), lr=lr_start)
bc100_sp_lr_sched = lr_scheduler.ReduceLROnPlateau(bc100_sp_optimizer, patience=6, verbose=True)
bc100_sp_valid_loss_max = np.Inf

ckpt_name = 'Bayesian/Models/bc100_sp.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    bc100_sp_net.load_state_dict(checkpoint['model_state_dict'])
    bc100_sp_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    bc100_sp_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    bc100_sp_valid_loss_max = checkpoint['valid_loss_max']
    print('Model loaded from {}'.format(ckpt_name))

for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc100_sp_train_loss, bc100_sp_train_acc, bc100_sp_train_kl = BCNN.train_model(bc100_sp_net, bc100_sp_optimizer, bc100_sp_criterion, c100_train_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_sp_valid_loss, bc100_sp_valid_acc = BCNN.validate_model(bc100_sp_net, bc100_sp_criterion, c100_valid_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_sp_lr_sched.step(bc100_sp_valid_loss)

    # save model if validation accuracy has increased
    if bc100_sp_valid_loss <= bc100_sp_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc100_sp_valid_loss_max, bc100_sp_valid_loss))
        th.save({
            'model_state_dict': bc100_sp_net.state_dict(),
            'optimizer_state_dict': bc100_sp_optimizer.state_dict(),
            'scheduler_state_dict': bc100_sp_lr_sched.state_dict(),
            'valid_loss_max': bc100_sp_valid_loss
        }, ckpt_name)
        bc100_sp_valid_loss_max = bc100_sp_valid_loss

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc100_sp_train_loss, bc100_sp_train_acc, bc100_sp_valid_loss, bc100_sp_valid_acc, bc100_sp_train_kl))

# After all epochs are complete, evaluate the model on the test set
bc100_sp_test_loss, bc100_sp_test_acc = BCNN.validate_model(bc100_sp_net, bc100_sp_criterion, c100_test_loader, num_ens=1, beta_type=beta_type)
print('Test Loss: {:.4f} \tTest Accuracy: {:.4f}'.format(bc100_sp_test_loss, bc100_sp_test_acc))

In [ ]:
# BayesianCNN with relu on CIFAR10
bc10_rl_net = BBBAlexNet(c10_outputs, c10_inputs, priors, activation_type='relu').to(device)
bc10_rl_criterion = metrics.ELBO(len(c10_trainset)).to(device)
bc10_rl_optimizer = Adam(bc10_rl_net.parameters(), lr=lr_start)
bc10_rl_lr_sched = lr_scheduler.ReduceLROnPlateau(bc10_rl_optimizer, patience=6, verbose=True)
bc10_rl_valid_loss_max = np.Inf

ckpt_name = 'Bayesian/Models/bc10_rl.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    bc10_rl_net.load_state_dict(checkpoint['model_state_dict'])
    bc10_rl_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    bc10_rl_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    bc10_rl_valid_loss_max = checkpoint['valid_loss_max']
    print('Model loaded from {}'.format(ckpt_name))

for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc10_rl_train_loss, bc10_rl_train_acc, bc10_rl_train_kl = BCNN.train_model(bc10_rl_net, bc10_rl_optimizer, bc10_rl_criterion, c10_train_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_rl_valid_loss, bc10_rl_valid_acc = BCNN.validate_model(bc10_rl_net, bc10_rl_criterion, c10_valid_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc10_rl_lr_sched.step(bc10_rl_valid_loss)

    # save model if validation accuracy has increased
    if bc10_rl_valid_loss <= bc10_rl_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc10_rl_valid_loss_max, bc10_rl_valid_loss))
        th.save({
            'model_state_dict': bc10_rl_net.state_dict(),
            'optimizer_state_dict': bc10_rl_optimizer.state_dict(),
            'scheduler_state_dict': bc10_rl_lr_sched.state_dict(),
            'valid_loss_max': bc10_rl_valid_loss
        }, ckpt_name)
        bc10_rl_valid_loss_max = bc10_rl_valid_loss
    

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc10_rl_train_loss, bc10_rl_train_acc, bc10_rl_valid_loss, bc10_rl_valid_acc, bc10_rl_train_kl))

# After all epochs are complete, evaluate the model on the test set
bc10_rl_test_loss, bc10_rl_test_acc = BCNN.validate_model(bc10_rl_net, bc10_rl_criterion, c10_test_loader, num_ens=1, beta_type=beta_type)
print('Test Loss: {:.4f} \tTest Accuracy: {:.4f}'.format(bc10_rl_test_loss, bc10_rl_test_acc))

In [ ]:
# BayesianCNN with relu on CIFAR100
bc100_rl_net = BBBAlexNet(c100_outputs, c100_inputs, priors, activation_type='relu').to(device)
bc100_rl_criterion = metrics.ELBO(len(c100_trainset)).to(device)
bc100_rl_optimizer = Adam(bc100_rl_net.parameters(), lr=lr_start)
bc100_rl_lr_sched = lr_scheduler.ReduceLROnPlateau(bc100_rl_optimizer, patience=6, verbose=True)
bc100_rl_valid_loss_max = np.Inf

ckpt_name = 'Bayesian/Models/bc100_rl.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    bc100_rl_net.load_state_dict(checkpoint['model_state_dict'])
    bc100_rl_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    bc100_rl_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    bc100_rl_valid_loss_max = checkpoint['valid_loss_max']
    print('Model loaded from {}'.format(ckpt_name))

for epoch in tqdm(range(n_epochs)):  # loop over the dataset multiple times

    bc100_rl_train_loss, bc100_rl_train_acc, bc100_rl_train_kl = BCNN.train_model(bc100_rl_net, bc100_rl_optimizer, bc100_rl_criterion, c100_train_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_rl_valid_loss, bc100_rl_valid_acc = BCNN.validate_model(bc100_rl_net, bc100_rl_criterion, c100_valid_loader, num_ens=1, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    bc100_rl_lr_sched.step(bc100_rl_valid_loss)

    # save model if validation accuracy has increased
    if bc100_rl_valid_loss <= bc100_rl_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            bc100_rl_valid_loss_max, bc100_rl_valid_loss))
        th.save({
            'model_state_dict': bc100_rl_net.state_dict(),
            'optimizer_state_dict': bc100_rl_optimizer.state_dict(),
            'scheduler_state_dict': bc100_rl_lr_sched.state_dict(),
            'valid_loss_max': bc100_rl_valid_loss
        }, ckpt_name)
        bc100_rl_valid_loss_max = bc100_rl_valid_loss

    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \ttrain_kl_div: {:.4f}'.format(
        epoch, bc100_rl_train_loss, bc100_rl_train_acc, bc100_rl_valid_loss, bc100_rl_valid_acc, bc100_rl_train_kl))

# After all epochs are complete, evaluate the model on the test set
bc100_rl_test_loss, bc100_rl_test_acc = BCNN.validate_model(bc100_rl_net, bc100_rl_criterion, c100_test_loader, num_ens=1, beta_type=beta_type)
print('Test Loss: {:.4f} \tTest Accuracy: {:.4f}'.format(bc100_rl_test_loss, bc100_rl_test_acc))

In [ ]:
# FrequentistCNN on CIFAR10
fc10_net = AlexNet(c10_outputs, c10_inputs).to(device)
fc10_criterion = nn.CrossEntropyLoss()
fc10_optimizer = Adam(fc10_net.parameters(), lr=lr_start)
fc10_lr_sched = lr_scheduler.ReduceLROnPlateau(fc10_optimizer, patience=6, verbose=True)
fc10_valid_loss_max = np.Inf

ckpt_name = 'Frequentist/Models/fc10.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    fc10_net.load_state_dict(checkpoint['model_state_dict'])
    fc10_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    fc10_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    fc10_valid_loss_max = checkpoint['valid_loss_max']
    print('Model loaded from {}'.format(ckpt_name))

for epoch in tqdm(range(n_epochs)):

    fc10_train_loss, fc10_train_acc = FCNN.train_model(fc10_net, fc10_optimizer, fc10_criterion, c10_train_loader)
    fc10_valid_loss, fc10_valid_acc = FCNN.validate_model(fc10_net, fc10_criterion, c10_valid_loader)
    fc10_lr_sched.step(fc10_valid_loss)

    # save model if validation accuracy has increased
    if fc10_valid_loss <= fc10_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            fc10_valid_loss_max, fc10_valid_loss))
        th.save({
            'model_state_dict': fc10_net.state_dict(),
            'optimizer_state_dict': fc10_optimizer.state_dict(),
            'scheduler_state_dict': fc10_lr_sched.state_dict(),
            'valid_loss_max': fc10_valid_loss
        }, ckpt_name)
        fc10_valid_loss_max = fc10_valid_loss
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc10_train_loss, fc10_train_acc, fc10_valid_loss, fc10_valid_acc))

# After all epochs are complete, evaluate the model on the test set
fc10_test_loss, fc10_test_acc = FCNN.validate_model(fc10_net, fc10_criterion, c10_test_loader)
print('Test Loss: {:.4f} \tTest Accuracy: {:.4f}'.format(fc10_test_loss, fc10_test_acc))

In [ ]:
# Frequentist CNN on CIFAR100
fc100_net = AlexNet(c100_outputs, c100_inputs).to(device)
fc100_criterion = nn.CrossEntropyLoss()
fc100_optimizer = Adam(fc100_net.parameters(), lr=lr_start)
fc100_lr_sched = lr_scheduler.ReduceLROnPlateau(fc100_optimizer, patience=6, verbose=True)
fc100_valid_loss_max = np.Inf

ckpt_name = 'Frequentist/Models/fc100.pth'
if os.path.isfile(ckpt_name):
    checkpoint = th.load(ckpt_name)
    fc100_net.load_state_dict(checkpoint['model_state_dict'])
    fc100_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    fc100_lr_sched.load_state_dict(checkpoint['scheduler_state_dict'])
    fc100_valid_loss_max = checkpoint['valid_loss_max']
    print('Model loaded from {}'.format(ckpt_name))

for epoch in tqdm(range(n_epochs)):

    fc100_train_loss, fc100_train_acc = FCNN.train_model(fc100_net, fc100_optimizer, fc100_criterion, c100_train_loader)
    fc100_valid_loss, fc100_valid_acc = FCNN.validate_model(fc100_net, fc100_criterion, c100_valid_loader)
    fc100_lr_sched.step(fc100_valid_loss)

    # save model if validation accuracy has increased
    if fc100_valid_loss <= fc100_valid_loss_max:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            fc100_valid_loss_max, fc100_valid_loss))
        th.save({
            'model_state_dict': fc100_net.state_dict(),
            'optimizer_state_dict': fc100_optimizer.state_dict(),
            'scheduler_state_dict': fc100_lr_sched.state_dict(),
            'valid_loss_max': fc100_valid_loss
        }, ckpt_name)
        fc100_valid_loss_max = fc100_valid_loss
            
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, fc100_train_loss, fc100_train_acc, fc100_valid_loss, fc100_valid_acc))

# After all epochs are complete, evaluate the model on the test set
fc100_test_loss, fc100_test_acc = FCNN.validate_model(fc100_net, fc100_criterion, c100_test_loader)
print('Test Loss: {:.4f} \tTest Accuracy: {:.4f}'.format(fc100_test_loss, fc100_test_acc))

In [ ]:
epochs = range(1, n_epochs+1)

plt.figure(figsize=(10,6))
plt.plot(epochs, bc10_sp_train_acc, label='bc10_sp')
plt.plot(epochs, bc100_sp_train_acc, label='bc100_sp')
plt.plot(epochs, bc10_rl_train_acc, label='bc10_rl')
plt.plot(epochs, bc100_rl_train_acc, label='bc100_rl')
plt.plot(epochs, fc10_train_acc, label='fc10')
plt.plot(epochs, fc100_train_acc, label='fc100')

plt.title('Comparison of training accuracies')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.show()

In [9]:
epochs = range(1, n_epochs+1)

plt.figure(figsize=(10,6))
plt.plot(epochs, bc10_sp_valid_acc, label='bc10_sp')
#plt.plot(epochs, bc100_sp_valid_acc, label='bc100_sp')
#plt.plot(epochs, bc10_rl_valid_acc, label='bc10_rl')
#plt.plot(epochs, bc100_rl_valid_acc, label='bc100_rl')
#plt.plot(epochs, fc10_valid_acc, label='fc10')
#plt.plot(epochs, fc100_valid_acc, label='fc100')

plt.title('Comparison of validation accuracies')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.show()

NameError: name 'bc10_sp_valid_acc' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
# Mislabel 10% of the CIFAR10 and CIFAR100 trainsets
c10_trainset_mislabel = data.mislabel_data(c10_trainset, c10_outputs, 0.1)
c100_trainset_mislabel = data.mislabel_data(c100_trainset, c100_outputs, 0.1)

# Create new dataloaders for the mislabeled datasets
c10_train_loader_mislabel, _, _ = data.getDataloader(
    c10_trainset_mislabel, c10_testset, valid_size, batch_size, num_workers)
c100_train_loader_mislabel, _, _ = data.getDataloader(
    c100_trainset_mislabel, c100_testset, valid_size, batch_size, num_workers)